<a href="https://colab.research.google.com/github/Psyclophe/datasets/blob/master/Semana2_1_Aps_Financieras5_Intro_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Aplicaciones-Financieras/blob/main/Semana2/1_Intro_Keras.ipynb)

# Dependencias

In [5]:
# Puede no ser necesaria si ya tienen instalado plotly
# !pip install -U plotly

In [6]:
!import pandas=='1.4.0'

/bin/bash: import: command not found


In [7]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branca                        0.6.0
CacheControl                  0.12.11
cached-property               1.5.2
cachetools                    5.3.0
cata

In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, mean_squared_error, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint
from keras import regularizers

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

import seaborn as sns

In [9]:
pio.templates.default = 'plotly_white'

# Datos

Utilizaremos los mismo datos de la clase pasada

https://www.kaggle.com/mlg-ulb/creditcardfraud/data


In [10]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Cruso-ApsFinancieras/semana3/creditcard.csv")
df

In [ ]:
# El resultado seria 'False' si NO hay valores nulos y sería 'True' si SI hay valores nulos
df.isnull().values.any()

In [ ]:
# 0: Normal
# 1: Fraudulento

print(df.Class.value_counts())
df.Class.value_counts() / len(df)

In [ ]:
df[df.Class == 0].Amount.describe()

In [ ]:
df[df.Class == 1].Amount.describe()

In [ ]:
df.loc[:, 'V1':'Amount']

In [ ]:
# Separación de características
X = df.iloc[:, 1:-1]
X

In [ ]:
# Variables objetivos
y = df.Class
y

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
normal_df = df[df.Class == 0]
fraud_df = df[df.Class == 1]

In [ ]:
bins = np.linspace(200, 2500, 100)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

ax = axes.ravel()

ax[0].hist(normal_df.Amount, bins, alpha=1, label='Normal')
ax[0].set_title('legitime', fontsize=20)

ax[1].hist(fraud_df.Amount, bins, alpha=1, label='Fraud')
ax[1].set_title('fraud', fontsize=20)
plt.show()

## División en conjuntos de entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10) # train_size=0.8

## Estandarización

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)

pd.DataFrame(X_train_norm)

"If you torture the data long enough, it will confess" Ronald H. Coase

# Modelo

## Configuración del modelo

In [ ]:
input_dim = X_train_norm.shape[1] # Número de columnas: 29
encoding_dim = 14
hidden_dim = int(encoding_dim / 2) 
learning_rate = 1e-5

In [ ]:
hidden_dim

In [ ]:
print(f'{1e-5: .9f}')

## Arquitectura de la red

In [ ]:
# Hiperparámetros muy importantes:
# - learning rate
# - batch size

In [ ]:
# Creación de las capas del modelo
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="relu", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder2 = Dense(hidden_dim, activation="relu")(encoder)

decoder = Dense(hidden_dim, activation='relu')(encoder2)
decoder_f = Dense(input_dim, activation='relu')(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder_f)

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.compile(loss='mean_squared_error',
                    optimizer='adam')

### Con las lineas de abajo pueden decidir que epoca de entrenamiento guardar

# cp = ModelCheckpoint(filepath="autoencoder_fraud.h5",
#                                save_best_only=True,
#                                verbose=0)

## Entrenamiento

In [ ]:
history = autoencoder.fit(X_train_norm, X_train_norm,
                          epochs=100,
                          batch_size=2048,
                          shuffle = True,
                          validation_split=.2,
                          verbose=1, 
                        #   callbacks = cp
                          ).history

In [ ]:
# Comando para recuperar los datos guardados del modelos, en este caso se guardaria la mejor epoca nada mas

# autoencoder = load_model('autoencoder_fraud.h5')

## Evaluación

In [ ]:
# Grafica de la pérdida del modelo en el tiempo
fig = go.Figure()
fig.add_trace(go.Scatter(y = history['loss'], name = 'loss'))
fig.add_trace(go.Scatter(y = history['val_loss'], name = 'val_loss'))
fig.update_layout(
    title = 'Pérdida del modelo',
    xaxis_title = 'Época (epoch)', 
    yaxis_title = 'Pérdida (MSE)'
)
fig.show()

In [ ]:
X_test_norm = scaler.transform(X_test)
X_test_pred = autoencoder.predict(X_test_norm)

# Error cuadratico medio, a mano
mse = mean_squared_error(X_test_norm.T, X_test_pred.T, multioutput = 'raw_values')
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': y_test})
error_df.describe()

In [ ]:
error_df

La precision y el recall son muy importantes, y a veces hay que buscar el modo de optimizarlos escogiendo un threshold adecuado.

El limite a escoger (threshold) depende de que se desea del modelo. Escoger si preferimos dejar pasar un fraude por etiquetarlo mal o si preferimos etiquetar mas como fraude e invertir en solucionarlos aunque no sean fraude.

In [ ]:
precision_rt, recall_rt, threshold_rt = precision_recall_curve(error_df.True_class, error_df.Reconstruction_error)
px.scatter(x = recall_rt, y = precision_rt, title = 'Precision vs. Recall', 
           labels = {
             'x': 'Recall', 
             'y': 'Precision'
           })

In [ ]:
len(threshold_rt)

In [ ]:
# Grafica de todos los limites (threshold)
plt.plot(threshold_rt[0:])
plt.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = threshold_rt, y = precision_rt[1:], name = "Precision"))
fig.add_trace(go.Scatter(x = threshold_rt, y = recall_rt[1:], name = "Recall"))

fig.update_layout(
    title = 'Precision y Recall para diferentes umbrales', 
    xaxis_title = 'Umbral (threshold)', 
    yaxis_title = 'Precision/Recall', 
    hovermode="x unified"
)

fig.show()

In [ ]:
threshold_fixed = 47

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
                    x = error_df[error_df.True_class == 0].index.to_numpy(), 
                    y = error_df[error_df.True_class == 0].Reconstruction_error, 
                    mode = 'markers', 
                    name = 'Normal'))

fig.add_trace(go.Scatter(
                    x = error_df[error_df.True_class == 1].index.to_numpy(), 
                    y = error_df[error_df.True_class == 1].Reconstruction_error, 
                    mode = 'markers', 
                    name = 'Fraude'))
fig.add_hline(threshold_fixed, annotation_text = 'Umbral fijo', line_dash = 'dash')

fig.update_layout(
    title = 'Error de reconstrucción para distintas clases', 
    yaxis_title = 'Error de Reconstrucción (MSE)', 
    xaxis_title = 'Índice del punto'
)
fig.show()

In [ ]:
pred_y = [1 if e > 47 else 0 for e in error_df.Reconstruction_error.values]
conf_matrix = confusion_matrix(pred_y, error_df.True_class)

plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, xticklabels=['Normal', 'Fraude'], yticklabels=['Normal', 'Fraude'], annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [ ]:
print(classification_report(y_test, pred_y, digits = 4))

# Ejercicios

- Agregar más capas al encoder y/o al decoder y comparar los resultados obtenidos. Agregar muchas capas al modelo puede hacer que se sobreajuste. Una manera de mitigarlo es agregando regularización o capas Droupout. Si considera que su modelo tiene sobreajuste agregue cualquiera de las dos o elimine capas.
- ¿Cuál es la utilidad de las funciones de activación? ¿Qué operaciones hacen las distintas [funciones de activación que tiene Keras](https://keras.io/api/layers/activations/)?
- Pruebe con diferentes funciones de activación y evalue los resultados. Las funciones de activación que tiene el modelo son tanh (tangente hiperbólica) y ReLU (Rectified Linear Unit).

# Ligas

- [Post Original](https://blogs.oracle.com/ai-and-datascience/post/fraud-detection-using-autoencoders-in-keras-with-a-tensorflow-backend)

- Info. sobre pulir hiperparámetros (batch_size, learn_rate, epochs,...)
  
  *   https://www.oreilly.com/library/view/hands-on-machine-learning/9781491962282/
  *   https://www.oreilly.com/library/view/natural-language-processing/9781484242674/

